In [1]:
import torch
import pandas as pd
import numpy as np

In [2]:
import os

In [3]:
os.getcwd()

'C:\\Users\\anves\\Workspace\\Learnings\\Deep Learning\\PyTorch'

# Basic Operations

In [5]:
x = torch.empty(5,3)

In [6]:
print(x)

tensor([[1.0194e-38, 8.4490e-39, 1.0469e-38],
        [9.3674e-39, 9.9184e-39, 8.7245e-39],
        [9.2755e-39, 8.9082e-39, 9.9184e-39],
        [8.4490e-39, 9.6429e-39, 1.0653e-38],
        [1.0469e-38, 4.2246e-39, 1.0378e-38]])


In [7]:
y = torch.ones(5,3)

In [8]:
print(y)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [9]:
z = torch.rand(5,3)

In [10]:
y + z

tensor([[1.8454, 1.4878, 1.9241],
        [1.3073, 1.3485, 1.4783],
        [1.3127, 1.4798, 1.8219],
        [1.3899, 1.0520, 1.1411],
        [1.3490, 1.6428, 1.1583]])

In [11]:
type(z)

torch.Tensor

In [12]:
torch.add(y,z)

tensor([[1.8454, 1.4878, 1.9241],
        [1.3073, 1.3485, 1.4783],
        [1.3127, 1.4798, 1.8219],
        [1.3899, 1.0520, 1.1411],
        [1.3490, 1.6428, 1.1583]])

In [13]:
y.add_(z)

tensor([[1.8454, 1.4878, 1.9241],
        [1.3073, 1.3485, 1.4783],
        [1.3127, 1.4798, 1.8219],
        [1.3899, 1.0520, 1.1411],
        [1.3490, 1.6428, 1.1583]])

In [14]:
y.numpy()

array([[1.8454473, 1.4878438, 1.9241104],
       [1.3073099, 1.3485458, 1.4783287],
       [1.3127446, 1.4797642, 1.8218685],
       [1.3898733, 1.0519794, 1.141073 ],
       [1.3490014, 1.642803 , 1.158284 ]], dtype=float32)

# Autograd - Automatic Differentiation

In [15]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [16]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [17]:
print(y.grad_fn)

In [18]:
y.requires_grad

True

In [20]:
z = y * y * y * 3
out = z.mean()

print(z, out)

tensor([[81., 81.],
        [81., 81.]], grad_fn=<MulBackward0>) tensor(81., grad_fn=<MeanBackward0>)


In [21]:
z.requires_grad

True

In [22]:
print(x.grad)

None


In [23]:
print(y.grad)

None


C:\Users\anves\Anaconda3\lib\site-packages\torch\_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:475.)
  return self._grad


In [24]:
out.backward()

In [25]:
print(x.grad)

tensor([[20.2500, 20.2500],
        [20.2500, 20.2500]])


# A MultiClass Classifier with PyTorch on Iris Dataset

### Read Data

In [26]:
data = pd.read_csv(os.getcwd()+'/iris.data', header = None)

In [27]:
data.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [28]:
len(data)

150

In [29]:
data[4] = data[4].map({'Iris-setosa' : 0, 'Iris-versicolor' : 1, 'Iris-virginica' : 2, })

In [30]:
data[4].value_counts()

0    50
1    50
2    50
Name: 4, dtype: int64

### Split the data into train, val, and test

In [58]:
Train = data.sample(frac=0.8,random_state=200)
test = data.drop(Train.index)

In [59]:
len(train)

108

In [60]:
len(test)

30

In [61]:
train = Train.sample(frac=0.9,random_state=200)

val = Train.drop(train.index)


In [62]:
len(train)

108

In [63]:
len(val)

12

In [64]:
train[4].value_counts()

0    38
1    36
2    34
Name: 4, dtype: int64

### Save them into separate files (It will help later)

In [65]:
train.to_csv('train.csv', index = False, header = False)
test.to_csv('test.csv', index = False, header = False)
val.to_csv('val.csv', index = False, header = False)

In [66]:
from torch.utils.data import Dataset

from torch.utils.data import DataLoader

from torchvision import transforms

### Create your dataset class

In [67]:
class IrisDataset(Dataset):
    
    def __init__(self, file_path, transform = None):
        self.data_frame = pd.read_csv(file_path, header = None)
        self.transform = transform
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self,idx):
        X = self.data_frame.iloc[idx, :4].values
        y = self.data_frame.iloc[idx,4]
        X = (X - X.mean())/X.std()
        
        return X, y
    
    

### Create dataset objects

In [68]:
train_dataset = IrisDataset('train.csv')
val_dataset = IrisDataset('val.csv')
test_dataset = IrisDataset('test.csv')

### Set Parameters

In [69]:
learning_rate = 0.01
batch_size = 20
epochs = 20

### Instantiate DataLoader objects

In [70]:
training_generator = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

val_generator = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

testing_generator = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

In [71]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

### Design your Neural Network

In [72]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3) 
        self.fc3 = nn.Linear(3, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=3, bias=True)
)


### Define loss criteria and optimizer

In [73]:
optimizer = optim.Adam(net.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

### Implement your training loop

In [74]:
net = Net()
net.train()
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(training_generator):
        inputs, labels = data
      #  print(labels)
        inputs, labels = (inputs).type(torch.FloatTensor), (labels)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(running_loss/batch_size)
    
    print('----------Epoch Complete---------')
    

0.33738970160484316
----------Epoch Complete---------
0.33649744987487795
----------Epoch Complete---------
0.3359237551689148
----------Epoch Complete---------
0.3353707790374756
----------Epoch Complete---------
0.3354958653450012
----------Epoch Complete---------
0.33769773244857787
----------Epoch Complete---------
0.33586044907569884
----------Epoch Complete---------
0.3380325198173523
----------Epoch Complete---------
0.3365610301494598
----------Epoch Complete---------
0.337666392326355
----------Epoch Complete---------
0.3367192089557648
----------Epoch Complete---------
0.3364183485507965
----------Epoch Complete---------
0.335911238193512
----------Epoch Complete---------
0.3377207159996033
----------Epoch Complete---------
0.3357671558856964
----------Epoch Complete---------
0.33595011830329896
----------Epoch Complete---------
0.33663920760154725
----------Epoch Complete---------
0.3368924558162689
----------Epoch Complete---------
0.3359149158000946
----------Epoch Complet

### Save your Model

In [78]:
torch.save(net, os.getcwd()+'/Simple_Net.pt')

### Load Model and Predict

In [79]:
Loaded_Net = torch.load(os.getcwd()+'/Simple_Net.pt')

In [80]:
correct = 0
total = 0
with torch.no_grad():
    for data in testing_generator:
        inputs, labels = data
        inputs = (inputs).type(torch.FloatTensor)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(predicted)

tensor([1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1])
tensor([1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2])
tensor([1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2])
tensor([1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1])
tensor([1, 1, 2, 1, 2, 1, 2, 2])
